In [25]:
import os 
import re
import time
import pandas as pd
import multiprocessing as mp
import numpy as np
import dask.dataframe as dd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import OrderedDict
import torch.nn.functional as F
from torch import nn
import math
import matplotlib.pyplot as plt

from statistics import mean

import torch
from torch import optim
import os

#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))
a=time.time()
path0 ='../' 
df = pd.read_csv(path0+"FinaldatasetN1.csv")

a=time.time()
#df=df[['channel','Next_Current_diff','Next_Pre_diff','size', 'packet_type','DataQueueLen','lossRate','SNext_Current_diff','SNext_Pre_diff','snir','throughput','Flow Bytes_s','Flow Packets_s','meanT_b_2P','minT_b_2P','maxT_b_2P','label']]     
#df=df[['channel','Next_Current_diff','Next_Pre_diff','size', 'packet_type','lossRate','SNext_Current_diff','SNext_Pre_diff','snir','throughput','Flow Bytes_s','Flow Packets_s','meanT_b_2P','minT_b_2P','maxT_b_2P','label']]     
#df=df[['channel','Next_Current_diff','Next_Pre_diff','size','snir','throughput','packet_type','SNext_Current_diff','SNext_Pre_diff','Flow Bytes_s','Flow Packets_s','meanT_b_2P','minT_b_2P','maxT_b_2P','label']]     
df=df[['channel','Next_Current_diff','Next_Pre_diff','size','packet_type','SNext_Current_diff','SNext_Pre_diff','snir','Flow Bytes_s','Flow Packets_s','meanT_b_2P','minT_b_2P','maxT_b_2P','label']]     


#df=df.to_numpy()
#df_Normal=df[200000:354961].copy()
df_Normal=df[0:354961].copy()

#print(df_Normal[0][1])
#print(df_Normal[-1][1])
df_UDP1=df[354961:763730].copy()
#print(df_UDP1[0][1])
#print(df_UDP1[-1][1])
df_UDP2=df[763730:1573000].copy()
#print(df_UDP2[0][1])
#print(df_UDP2[-1][1])
#df_jam1=df[1573000:1696992].copy()
df_jam1=df[1573000:1696992].copy()

#print(df_jam1[0][1])
#print(df_jam1[-1][1])
df_jam2=df[1696992:2714727].copy()
#print(df_jam2[0][1])
#print(df_jam2[-1][1])
df_pluies=df[2714727:-1].copy()
#print(df_pluies[0][1])
#print(df_pluies[-1][1])
del(df)

X=df_Normal.drop(columns = ['label']).copy()
y=df_Normal[['label']].copy()
#print(y.columns)
X1=df_UDP1.drop(columns = ['label']).copy()
y1=df_UDP1[['label']].copy()
X2=df_UDP2.drop(columns = ['label']).copy()
y2=df_UDP2[['label']].copy()
X3=df_jam1.drop(columns = ['label']).copy()
y3=df_jam1[['label']].copy()
X4=df_jam2.drop(columns = ['label']).copy()
y4=df_jam2[['label']].copy()
X5=df_pluies.drop(columns = ['label']).copy()
y5=df_pluies[['label']].copy()

train=0.7
#train=0.01

del(df_Normal)
del(df_UDP1)
del(df_UDP2)
del(df_jam1)
del(df_jam2)
del(df_pluies)


xcol=X.columns
ycol=y.columns


X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=train,shuffle=False)
X_train1, X_rem1, y_train1, y_rem1 = train_test_split(X1,y1, train_size=train,shuffle=False)
X_train2, X_rem2, y_train2, y_rem2 = train_test_split(X2,y2, train_size=train,shuffle=False)
X_train3, X_rem3, y_train3, y_rem3 = train_test_split(X3,y3, train_size=train,shuffle=False)
X_train4, X_rem4, y_train4, y_rem4 = train_test_split(X4,y4, train_size=train,shuffle=False)
X_train5, X_rem5, y_train5, y_rem5 = train_test_split(X5,y5, train_size=train,shuffle=False)

#validation=0.5
#X_valid, X_rem, y_valid, y_rem = train_test_split(X_rem,y_rem, test_size=validation,shuffle=False)
#X_valid1, X_rem1, y_valid1, y_rem1 = train_test_split(X_rem1,y_rem1, test_size=validation,shuffle=False)
#X_valid2, X_rem2, y_valid2, y_rem2 = train_test_split(X_rem2,y_rem2, test_size=validation,shuffle=False)
#X_valid3, X_rem3, y_valid3, y_rem3 = train_test_split(X_rem3,y_rem3, test_size=validation,shuffle=False)
#X_valid4, X_rem4, y_valid4, y_rem4= train_test_split(X_rem4,y_rem4, test_size=validation,shuffle=False)
#X_valid5, X_rem5, y_valid5, y_rem5 = train_test_split(X_rem5,y_rem5, test_size=validation,shuffle=False)

#validation=0.65
#X_valid, X_rem, y_valid, y_rem = train_test_split(X_rem,y_rem, test_size=validation,shuffle=False)
#X_valid1, X_rem1, y_valid1, y_rem1 = train_test_split(X_rem1,y_rem1, test_size=validation,shuffle=False)
#X_valid2, X_rem2, y_valid2, y_rem2 = train_test_split(X_rem2,y_rem2, test_size=validation,shuffle=False)
#X_valid3, X_rem3, y_valid3, y_rem3 = train_test_split(X_rem3,y_rem3, test_size=validation,shuffle=False)
#X_valid4, X_rem4, y_valid4, y_rem4= train_test_split(X_rem4,y_rem4, test_size=validation,shuffle=False)
#X_valid5, X_rem5, y_valid5, y_rem5 = train_test_split(X_rem5,y_rem5, test_size=validation,shuffle=False)



X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.57,shuffle=False)
X_valid1, X_test1, y_valid1, y_test1 = train_test_split(X_rem1,y_rem1, test_size=0.57,shuffle=False)
X_valid2, X_test2, y_valid2, y_test2 = train_test_split(X_rem2,y_rem2, test_size=0.57,shuffle=False)
X_valid3, X_test3, y_valid3, y_test3 = train_test_split(X_rem3,y_rem3, test_size=0.57,shuffle=False)
X_valid4, X_test4, y_valid4, y_test4 = train_test_split(X_rem4,y_rem4, test_size=0.57,shuffle=False)
X_valid5, X_test5, y_valid5, y_test5 = train_test_split(X_rem5,y_rem5, test_size=0.57,shuffle=False)


X_train=np.concatenate((X_train, X_train1, X_train2,X_train3, X_train4,X_train5))
X_valid=np.concatenate((X_valid, X_valid1, X_valid2,X_valid3, X_valid4,X_valid5))
X_test=np.concatenate((X_test, X_test1, X_test2,X_test3, X_test4,X_test5))

y_train=np.concatenate((y_train, y_train1, y_train2,y_train3, y_train4,y_train5))
y_valid=np.concatenate((y_valid, y_valid1, y_valid2,y_valid3, y_valid4,y_valid5))
y_test=np.concatenate((y_test, y_test1, y_test2,y_test3, y_test4,y_test5))

X_train = pd.DataFrame(X_train, columns= xcol)
X_valid = pd.DataFrame(X_valid, columns= xcol)
X_test = pd.DataFrame(X_test, columns= xcol)

y_train = pd.DataFrame(y_train, columns= ycol)
y_valid = pd.DataFrame(y_valid, columns= ycol)
y_test = pd.DataFrame(y_test, columns= ycol)

#########333




In [2]:



a-time.time()
X_train1=X_train.values
y_train1=y_train.values
#X_train1=X_train1.astype(float)

torch_tensor = torch.tensor(X_train1)
#torch_tensor[torch_tensor != torch_tensor]=float(0)
t=[]
test=0
test0=0

for j in y_train1:
    i=j[0]
#    print(i)
    if (i=='Normal'):
        t.append(int(0))
        test0+=1
    #t.append(int(1))
    if (i=='DDOS_UDP_FLOOD'):
        t.append(int(1))
    if (i=='PLUIES_ET_ORAGES'):
        t.append(int(2))
        test+=1
    if (i=='BROUILLAGE_Trafic'):
        t.append(int(3))
print(test)
labels = torch.LongTensor(t)
#print(torch_tensor[-1])
#print(labels[-1])
print(labels)
#print(df_label)
#print(labels.shape)

#df00 = pd.read_csv(path0+"small/testbed3/Dataset_S03Filtrer.csv")
#bigX = df00[['Next_Current_diff','Next_Pre_diff','SNext_Current_diff','SNext_Pre_diff','rcvdPK','duration(ms)','droppedPKWrongPort','sentPK','size','channel','throughput']]         
#bigY = df00['label']
bigX = X_valid
#[['Next_Current_diff','Next_Pre_diff','SNext_Current_diff','SNext_Pre_diff','rcvdPK','duration(ms)','droppedPKWrongPort','sentPK','size','throughput']]  ,'channel'       
bigY = y_valid['label']
#print(bigY)
X_valid1=bigX.values
y_valid1=bigY.values

#X_valid1=X_test.values
#y_valid1=y_test.values
#X_train1=X_train1.astype(float)

vtorch_tensor = torch.tensor(X_valid1)
v=[]

for i in y_valid1:
    #i=j[0]
    #print(i)
    if (i=='Normal'):
        v.append(int(0))
        #t.append(int(1))
    if (i=='DDOS_UDP_FLOOD'):
        v.append(int(1))
    if (i=='PLUIES_ET_ORAGES'):
        v.append(int(2))
    if (i=='BROUILLAGE_Trafic'):
        v.append(int(3))

vlabels = torch.LongTensor(v)


X_test1=X_test.values
y_test1=y_test.values

#X_train1=X_train1.astype(float)

ttorch_tensor = torch.tensor(X_test1)

#torch_tensor[torch_tensor != torch_tensor]=float(0)
t=[]

for j in y_test1:
    i=j[0]
#    print(i)
    if (i=='Normal'):
        t.append(int(0))
        #t.append(int(1))
    if (i=='DDOS_UDP_FLOOD'):
        t.append(int(1))
    if (i=='PLUIES_ET_ORAGES'):
        t.append(int(2))
    if (i=='BROUILLAGE_Trafic'):
        t.append(int(3))

tlabels = torch.LongTensor(t)
#print(torch_tensor[-1])
#print(labels[-1])
print(len(labels))
print(len(vlabels))
print(len(tlabels))

del(X)
del(y)
del(X1)
del(y1)
del(X2)
del(y2)
del(X3)
del(y3)
del(X4)
del(y4)
del(X5)
del(y5)
del(X_train1); del(X_train2);del(X_train3); del(X_train4);del(X_train5)
del(y_train1); del(y_train2);del(y_train3); del(y_train4);del(y_train5)
del(y_rem);del(y_rem1); del(y_rem2);del(y_rem3); del(y_rem4);del(y_rem5)
del(X_rem);del(X_rem1); del(X_rem2);del(X_rem3); del(X_rem4);del(X_rem5)

#X_rem1, y_train1, y_rem1
del(X_train)
del(X_valid)
del(X_valid1); del(X_valid2);del(X_valid3); del(X_valid4);del(X_valid5)
del(y_valid1); del(y_valid2);del(y_valid3); del(y_valid4);del(y_valid5)


del(X_test)
del(X_test1); del(X_test2);del(X_test3); del(X_test4);del(X_test5)
del(y_test1); del(y_test2);del(y_test3); del(y_test4);del(y_test5)

del(y_train)
del(y_valid)
del(y_test)
del(t)
del(v)

inputs=torch_tensor[:,:] 
labels1=labels[:]
del(torch_tensor)
del(labels)
#print(len(inputs))
#print(len(labels1))
vinputs =vtorch_tensor[:,:]
vlabels1=vlabels[:]
del(vtorch_tensor)
del(vlabels)
tinputs =ttorch_tensor[:,:] 
tlabels1=tlabels[:]
del(ttorch_tensor)
del(tlabels)




263116
tensor([0, 0, 0,  ..., 0, 0, 0])
2381331
438845
581729
tensor(1598574)
tensor(289624)
tensor(388668)
21.27530026435852


In [3]:
torch.set_default_dtype(torch.float64)
USE_GPU = True

from torch import nn, optim
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
# input_size = 784 # 28x28
num_classes = 4
#num_epochs = 2
batch_size = 200
learning_rate = 0.01

input_size = 13
sequence_length = 28
hidden_size = 132
num_layers = 2

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers,batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)
        
        # or:
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        x=x.view(-1,1,13)
        #print(x.shape)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        #c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        # x: (n, 28, 28), h0: (2, n, 128)
        
        # Forward propagate RNN
        out, _ = self.rnn(x, h0)  
        # or:
        #out, _ = self.lstm(x, (h0,c0))  
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
         
        #out = self.fc(out)
        x = F.log_softmax(self.fc(out), dim=1)
        # out: (n, 10)
        return x

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

#criterion = nn.NLLLoss()
Fonction_de_perte = nn.NLLLoss()

#optimizer = optim.SGD(model.parameters(), lr=0.003)
optimizer = optim.Adam(model.parameters(), lr=0.003)

model.load_state_dict(torch.load('models\\dataset2_1000_200_RNN.pth'))

#indices = [0,3, 299:303]

<All keys matched successfully>

In [32]:
import os 
import re
import time
import pandas as pd
import multiprocessing as mp
import numpy as np
import dask.dataframe as dd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import OrderedDict
import torch.nn.functional as F
from torch import nn
import math
import matplotlib.pyplot as plt

from statistics import mean

import torch
from torch import optim
import os

#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))
a=time.time()
path0 ='../' 
df = pd.read_csv(path0+"FinaldatasetN1.csv")
print('improt time '+str(time.time()-a))

a=time.time()
#df=df[['channel','Next_Current_diff','Next_Pre_diff','size', 'packet_type','DataQueueLen','lossRate','SNext_Current_diff','SNext_Pre_diff','snir','throughput','Flow Bytes_s','Flow Packets_s','meanT_b_2P','minT_b_2P','maxT_b_2P','label']]     
#df=df[['channel','Next_Current_diff','Next_Pre_diff','size', 'packet_type','lossRate','SNext_Current_diff','SNext_Pre_diff','snir','throughput','Flow Bytes_s','Flow Packets_s','meanT_b_2P','minT_b_2P','maxT_b_2P','label']]     
#df=df[['channel','Next_Current_diff','Next_Pre_diff','size','snir','throughput','packet_type','SNext_Current_diff','SNext_Pre_diff','Flow Bytes_s','Flow Packets_s','meanT_b_2P','minT_b_2P','maxT_b_2P','label']]     
df=df[['channel','Next_Current_diff','Next_Pre_diff','size','packet_type','SNext_Current_diff','SNext_Pre_diff','snir','Flow Bytes_s','Flow Packets_s','meanT_b_2P','minT_b_2P','maxT_b_2P','label']]     


#df=df.to_numpy()
#df_Normal=df[200000:354961].copy()
df_Normal=df[0:354961].copy()

#print(df_Normal[0][1])
#print(df_Normal[-1][1])
df_UDP1=df[354961:763730].copy()
#print(df_UDP1[0][1])
#print(df_UDP1[-1][1])
df_UDP2=df[763730:1573000].copy()
#print(df_UDP2[0][1])
#print(df_UDP2[-1][1])
#df_jam1=df[1573000:1696992].copy()
df_jam1=df[1573000:1696992].copy()

#print(df_jam1[0][1])
#print(df_jam1[-1][1])
df_jam2=df[1696992:2714727].copy()
#print(df_jam2[0][1])
#print(df_jam2[-1][1])
df_pluies=df[2714727:-1].copy()
#print(df_pluies[0][1])
#print(df_pluies[-1][1])
del(df)

X=df_Normal.drop(columns = ['label']).copy()
y=df_Normal[['label']].copy()
#print(y.columns)
X1=df_UDP1.drop(columns = ['label']).copy()
y1=df_UDP1[['label']].copy()
X2=df_UDP2.drop(columns = ['label']).copy()
y2=df_UDP2[['label']].copy()
X3=df_jam1.drop(columns = ['label']).copy()
y3=df_jam1[['label']].copy()
X4=df_jam2.drop(columns = ['label']).copy()
y4=df_jam2[['label']].copy()
X5=df_pluies.drop(columns = ['label']).copy()
y5=df_pluies[['label']].copy()

train=0.7
#train=0.01

del(df_Normal)
del(df_UDP1)
del(df_UDP2)
del(df_jam1)
del(df_jam2)
del(df_pluies)


xcol=X.columns
ycol=y.columns


X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=train,shuffle=False)
X_train1, X_rem1, y_train1, y_rem1 = train_test_split(X1,y1, train_size=train,shuffle=False)
X_train2, X_rem2, y_train2, y_rem2 = train_test_split(X2,y2, train_size=train,shuffle=False)
X_train3, X_rem3, y_train3, y_rem3 = train_test_split(X3,y3, train_size=train,shuffle=False)
X_train4, X_rem4, y_train4, y_rem4 = train_test_split(X4,y4, train_size=train,shuffle=False)
X_train5, X_rem5, y_train5, y_rem5 = train_test_split(X5,y5, train_size=train,shuffle=False)

#validation=0.5
#X_valid, X_rem, y_valid, y_rem = train_test_split(X_rem,y_rem, test_size=validation,shuffle=False)
#X_valid1, X_rem1, y_valid1, y_rem1 = train_test_split(X_rem1,y_rem1, test_size=validation,shuffle=False)
#X_valid2, X_rem2, y_valid2, y_rem2 = train_test_split(X_rem2,y_rem2, test_size=validation,shuffle=False)
#X_valid3, X_rem3, y_valid3, y_rem3 = train_test_split(X_rem3,y_rem3, test_size=validation,shuffle=False)
#X_valid4, X_rem4, y_valid4, y_rem4= train_test_split(X_rem4,y_rem4, test_size=validation,shuffle=False)
#X_valid5, X_rem5, y_valid5, y_rem5 = train_test_split(X_rem5,y_rem5, test_size=validation,shuffle=False)

#validation=0.65
#X_valid, X_rem, y_valid, y_rem = train_test_split(X_rem,y_rem, test_size=validation,shuffle=False)
#X_valid1, X_rem1, y_valid1, y_rem1 = train_test_split(X_rem1,y_rem1, test_size=validation,shuffle=False)
#X_valid2, X_rem2, y_valid2, y_rem2 = train_test_split(X_rem2,y_rem2, test_size=validation,shuffle=False)
#X_valid3, X_rem3, y_valid3, y_rem3 = train_test_split(X_rem3,y_rem3, test_size=validation,shuffle=False)
#X_valid4, X_rem4, y_valid4, y_rem4= train_test_split(X_rem4,y_rem4, test_size=validation,shuffle=False)
#X_valid5, X_rem5, y_valid5, y_rem5 = train_test_split(X_rem5,y_rem5, test_size=validation,shuffle=False)



X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.57,shuffle=False)
X_valid1, X_test1, y_valid1, y_test1 = train_test_split(X_rem1,y_rem1, test_size=0.57,shuffle=False)
X_valid2, X_test2, y_valid2, y_test2 = train_test_split(X_rem2,y_rem2, test_size=0.57,shuffle=False)
X_valid3, X_test3, y_valid3, y_test3 = train_test_split(X_rem3,y_rem3, test_size=0.57,shuffle=False)
X_valid4, X_test4, y_valid4, y_test4 = train_test_split(X_rem4,y_rem4, test_size=0.57,shuffle=False)
X_valid5, X_test5, y_valid5, y_test5 = train_test_split(X_rem5,y_rem5, test_size=0.57,shuffle=False)


X_train=np.concatenate((X_train, X_train1, X_train2,X_train3, X_train4,X_train5))
X_valid=np.concatenate((X_valid, X_valid1, X_valid2,X_valid3, X_valid4,X_valid5))
X_test=np.concatenate((X_test, X_test1, X_test2,X_test3, X_test4,X_test5))

y_train=np.concatenate((y_train, y_train1, y_train2,y_train3, y_train4,y_train5))
y_valid=np.concatenate((y_valid, y_valid1, y_valid2,y_valid3, y_valid4,y_valid5))
y_test=np.concatenate((y_test, y_test1, y_test2,y_test3, y_test4,y_test5))

X_train = pd.DataFrame(X_train, columns= xcol)
X_valid = pd.DataFrame(X_valid, columns= xcol)
X_test = pd.DataFrame(X_test, columns= xcol)

y_train = pd.DataFrame(y_train, columns= ycol)
y_valid = pd.DataFrame(y_valid, columns= ycol)
y_test = pd.DataFrame(y_test, columns= ycol)

#########333



def Test():  


    

    from torch import nn, optim
    import torch.nn.functional as F

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Hyper-parameters 
    # input_size = 784 # 28x28
    num_classes = 4
    #num_epochs = 2
    batch_size = 200
    learning_rate = 0.01

    input_size = 13
    sequence_length = 28
    hidden_size = 132
    num_layers = 2

    class RNN(nn.Module):
        def __init__(self, input_size, hidden_size, num_layers, num_classes):
            super(RNN, self).__init__()
            self.num_layers = num_layers
            self.hidden_size = hidden_size
            self.rnn = nn.RNN(input_size, hidden_size, num_layers,batch_first=True)
            # -> x needs to be: (batch_size, seq, input_size)

            # or:
            #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
            #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
            self.fc = nn.Linear(hidden_size, num_classes)

        def forward(self, x):
            # Set initial hidden states (and cell states for LSTM)
            x=x.view(-1,1,13)
            #print(x.shape)
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
            #c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 

            # x: (n, 28, 28), h0: (2, n, 128)

            # Forward propagate RNN
            out, _ = self.rnn(x, h0)  
            # or:
            #out, _ = self.lstm(x, (h0,c0))  

            # out: tensor of shape (batch_size, seq_length, hidden_size)
            # out: (n, 28, 128)

            # Decode the hidden state of the last time step
            out = out[:, -1, :]
            # out: (n, 128)

            #out = self.fc(out)
            x = F.log_softmax(self.fc(out), dim=1)
            # out: (n, 10)
            return x

    model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

    #criterion = nn.NLLLoss()
    Fonction_de_perte = nn.NLLLoss()

    #optimizer = optim.SGD(model.parameters(), lr=0.003)
    optimizer = optim.Adam(model.parameters(), lr=0.003)

    model.load_state_dict(torch.load('models\\dataset2_1000_100_RNN.pth'))

    #indices = [0,3, 299:303]



   

    path0 ='D:\\share\\Data_file\\Senarios\\new_test\\dataset2.csv' 
    df = pd.read_csv(path0)
    df1=df[['channel','Next_Current_diff','Next_Pre_diff','size','packet_type','SNext_Current_diff','SNext_Pre_diff','snir','Flow Bytes_s','Flow Packets_s','meanT_b_2P','minT_b_2P','maxT_b_2P']]     
    np_data=df.to_numpy()
    df1=df1.to_numpy()
    ttorch_tensor = torch.tensor(df1)
    tinputs =ttorch_tensor[:,:] 
    output = torch.exp(model(tinputs))
    #test_loss=Fonction_de_perte(output, tlabels1)
    top_p , top_c = output.topk(1, dim=1)
    node=set()
    node1=set()
    node2=dict()
    node10=dict()
    node11=set()


    #with open(location+"results/index.txt", "w") as f:
     #   #f.write(str(C_index+1))
      #  f.close()
    name='Pluies'
    if(name=='Pluies'):
        print('pluies')
        print(len(top_c))
        top_c = top_c.cpu().detach().numpy()
        for index,i in enumerate(top_c):
            i=i[0]
            i1=np_data[index][5]+' '+np_data[index][6]

            if (i1 in node11):
                    pass
                    node10[i1]+=1
                    #print(node2[i]) 
            else:
                    node11.add(i1)
                    #[sentTime , n]
                    node10[i1]=1
            if(i==2):
                #print('udp attack from '+str(np_data[index][5])+' and with probability of '+str(top_p[index]))
                i1=np_data[index][5]+' '+np_data[index][6]
                if (i1 in node):
                    pass
                    node2[i1][1]+=1
                    #print(node2[i]) 
                else:
                    node.add(i1)
                    #[sentTime , n]
                    node2[i1]=list([np_data[index][1],1])

        somme=0
        max1=0
        for i in node2:
            node2[i].append(node2[i][1]*100/len(top_c))
            somme+=node2[i][1]*100/len(top_c)
            if(max1<(node2[i][1]*100/len(top_c))):
                max1=node2[i][1]*100/len(top_c)
        if(somme>40 or 1):

            print('=========================================================================================================')
            print('                                   Pluies et Orages (Brouillage Naturel)')
            print('=========================================================================================================')
            #now = datetime.now()
            #print(' Detection entre :'+str(dt_0)+' et ' + str(now.strftime(" %H:%M:%S")))
            for i in node2:
                print('Flux entre '+str(i.replace(' ', ' et '))+' avec une probabilite de '+str("{:.2f}".format(node2[i][1]*100/node10[i])) + '% ( '+ str("{:.2f}".format(node2[i][2])) +' % de flux capture )'  )
    node=set()
    node1=set()
    node2=dict()
    node10=dict()
    node11=set()

    name='Jam'
    if(name=='Jam'):
        #print(location)
        #print(np_data[0])
        for index,i in enumerate(top_c):
            i=i[0]
            i1=np_data[index][5]+' '+np_data[index][6]
            if (i1 in node11):
                    pass
                    node10[i1]+=1
                    #print(node2[i]) 
            else:
                    node11.add(i1)
                    #[sentTime , n]
                    node10[i1]=1
            if(i==3):

                #print('udp attack from '+str(np_data[index][5])+' and with probability of '+str(top_p[index]))
                i1=np_data[index][5]+' '+np_data[index][6]
                if (i1 in node):
                    pass
                    node2[i1][1]+=1
                    #print(node2[i]) 
                else:
                    node.add(i1)
                    #[sentTime , n]
                    node2[i1]=list([np_data[index][1],1])
        somme=0
        max1=0
        for i in node2:
            node2[i].append(node2[i][1]*100/len(top_c))
            somme+=node2[i][1]*100/len(top_c)
            if(max1<(node2[i][1]*100/len(top_c))):
                max1=node2[i][1]*100/len(top_c)
        if(somme>35 or 1):

            print('=========================================================================================================')
            print('                                              Jamming')
            print('=========================================================================================================')
           # now = datetime.now()
            #print(' Detection entre :'+str(dt_0)+' et ' + str(now.strftime(" %H:%M:%S")))

            for i in node2:
                print('Flux entre '+str(i.replace(' ', ' et '))+' avec une probabilite de '+str("{:.2f}".format(node2[i][1]*100/node10[i])) + ' % ( '+ str("{:.2f}".format(node2[i][2])) +' % de flux capture )'  )
    node=set()
    node1=set()
    node2=dict()
    node10=dict()
    node11=set()

    name='Flood'
    if(name=='Flood'):
        #print('flood')
        #top_c = top_c.cpu().detach().numpy()
        for index,i in enumerate(top_c):
            i=i[0]
            i1=np_data[index][5]+' '+np_data[index][6]

            if (i1 in node11):
                    pass
                    node10[i1]+=1
                    #print(node2[i]) 
            else:
                    node11.add(i1)
                    #[sentTime , n]
                    node10[i1]=1
            if(i==1):
                #print('udp attack from '+str(np_data[index][5])+' and with probability of '+str(top_p[index]))
                i1=np_data[index][5]+' '+np_data[index][6]
                if (i1 in node):
                    pass
                    node2[i1][1]+=1
                    #print(node2[i]) 
                else:
                    node.add(i1)
                    #[sentTime , n]
                    node2[i1]=list([np_data[index][1],1])

        somme=0
        max1=0
        for i in node2:
            node2[i].append(node2[i][1]*100/len(top_c))
            somme+=node2[i][1]*100/len(top_c)
            if(max1<(node2[i][1]*100/len(top_c))):
                max1=node2[i][1]*100/len(top_c)
        del(top_c)
        if(somme>20 or 1):

            print('=========================================================================================================')
            print('                                      Attaque par Flood')
            print('=========================================================================================================')

            #now = datetime.now()
            #print(' Detection entre :'+str(dt_0)+' et ' + str(now.strftime(" %H:%M:%S")))

            for i in node2:
                print('Flux entre '+str(i.replace(' ', ' et '))+' avec une probabilite de '+str("{:.2f}".format(node2[i][1]*100/node10[i])) + '% ( '+ str("{:.2f}".format(node2[i][2])) +' % de flux capture )'  )


improt time 26.086262464523315


In [ ]:
###################################

In [ ]:
###################################

In [ ]:
###################################

In [ ]:
###################################

In [ ]:
###################################

In [ ]:
###################################

In [34]:
Test()

pluies
38100
                                   Pluies et Orages (Brouillage Naturel)
Flux entre 19.10.1.104 et 19.10.1.107 avec une probabilite de 100.00% ( 4.62 % de flux capture )
Flux entre 19.10.1.103 et 19.11.20.100 avec une probabilite de 100.00% ( 6.51 % de flux capture )
Flux entre 19.10.1.104 et 19.10.1.106 avec une probabilite de 100.00% ( 1.50 % de flux capture )
                                              Jamming
Flux entre 19.10.1.101 et 19.11.19.100 avec une probabilite de 100.00 % ( 0.78 % de flux capture )
Flux entre 19.10.1.102 et 19.11.16.100 avec une probabilite de 100.00 % ( 0.80 % de flux capture )
Flux entre 19.10.1.102 et 19.11.14.100 avec une probabilite de 100.00 % ( 0.80 % de flux capture )
Flux entre 19.10.1.101 et 19.11.16.100 avec une probabilite de 100.00 % ( 3.62 % de flux capture )
Flux entre 19.10.1.102 et 19.10.1.105 avec une probabilite de 100.00 % ( 1.53 % de flux capture )
Flux entre 19.10.1.100 et 19.10.1.108 avec une probabilite de 100.00 % ( 1